## 커스텀 평가지표 정의하여 사용하기

In [ ]:
from tensorflow.keras import backend as K

# 커스텀 평가지표를 정의합니다.
def recall_metric(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0.0, 1.0)))
    pred_pos = K.sum(K.round(K.clip(y_true, 0.0, 1.0)))
    recall = true_pos / (pred_pos + K.epsilon())
        
    return recall

def precision_metric(y_true, y_pred):
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0.0, 1.0)))
    pred_pos = K.sum(K.round(K.clip(y_pred, 0.0, 1.0)))
    precision = true_pos / (pred_pos + K.epsilon())
        
    return precision

def f1_metric(y_true, y_pred):
    recall = recall_metric(y_true, y_pred)
    precision = precision_metric(y_true, y_pred)
    
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, random_state = 777)

x_train = (x_train.reshape(-1, 28, 28, 1))
x_val = (x_val.reshape(-1, 28, 28, 1))
x_test = (x_test.reshape(-1, 28, 28, 1))

ce_layer = tf.keras.layers.CategoryEncoding(num_tokens=np.unique(y_train).__len__(),
                                 output_mode="one_hot")

y_train = ce_layer(list(y_train))
y_val = ce_layer(list(y_val))
y_test = ce_layer(list(y_test))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Rescaling
from tensorflow.keras.layers import Input

inputs = Input(shape = (28, 28, 1))

x = Rescaling(scale = 1./255)(inputs)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = Conv2D(32, (3, 3), activation = 'relu')(x)
x = MaxPooling2D(strides = 2)(x)

x = GlobalAveragePooling2D()(x)
x = Dense(10, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = x)

# 정의한 평가 지표를 사용합니다.
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy',
              metrics = ['acc', recall_metric, precision_metric, f1_metric])

model.fit(x_train, y_train,
         batch_size = 32,
         validation_data = (x_val, y_val),
         epochs = 10)